In [85]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, ZeroPadding2D
from tensorflow.keras import models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as pyplot
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageFilter
from urllib import request

In [68]:
# PROBLEM 1
# Model from the assignment
model = models.Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# Show the summary
model.summary()

Model: &quot;sequential_22&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [69]:
# In the summary above, we see three columns: the Layer (type), the Output Shape, and the Param #. In order to calculate the final column, we will also need the number of filters and the kernel size, which we will pull directly from the code for the model above. Generally speaking, in neural network architecture the formula for learnable params is: (inputs * ouputs) + biases. For CNNs, the inputs are the number of filters of the previous layer (if prev layer is dense, it is the number of nodes), the outputs are the number of filters in the current layer * size of those filters, and the bias is the number of filters in the current layer.

# So, in the model summary above, the Param # for each layer is as follows:
# 1. (32 * 9) + 32 = 320
# 2. 0 because max pooling is functional and has no learnable params
# 3. (32 * 64 * 9) + 64 = 18496
# 4. 0 because max pooling is functional and has no learnable params
# 5. (64 * 64 * 9) + 64 = 36928

In [70]:
# Import data from mnist using code from assignment
# Split into train images/labels and test images/labels
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape train/test images to include an explicit channel
# Convert train/test images to float32 type for use in CNN
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# Convert to categorical using keras utility function
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [71]:
# PROBLEM 2
# Define model with maxpooling layers, based on code from assignment
# Add maxpooling layer after each conv2d layers

# Define number of epochs for use throughout the assignment
num_epochs = 5

max_model = models.Sequential([
    Conv2D( 32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
# Compile and fit model according to assignment instructions, retaining history for later recall
max_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
max_history = max_model.fit(train_images, train_labels, epochs=num_epochs, batch_size=64)

Epoch 1/5
938/938 [==============================] - 19s 20ms/step - loss: 0.1789 - accuracy: 0.9443
Epoch 2/5
938/938 [==============================] - 18s 20ms/step - loss: 0.0470 - accuracy: 0.9857
Epoch 3/5
938/938 [==============================] - 19s 20ms/step - loss: 0.0326 - accuracy: 0.9899
Epoch 4/5
938/938 [==============================] - 19s 20ms/step - loss: 0.0247 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 18s 20ms/step - loss: 0.0190 - accuracy: 0.9942


In [72]:
# Define model with average pooling, based on code from assignment
# Add avgpooling layer after each conv2d layers
avg_model = models.Sequential([
    Conv2D( 32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    AveragePooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    AveragePooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
# Compile and fit model according to assignment instructions, retaining history for later recall
avg_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
avg_history = avg_model.fit(train_images, train_labels, epochs=num_epochs, batch_size=64)

Epoch 1/5
938/938 [==============================] - 18s 19ms/step - loss: 0.2146 - accuracy: 0.9327
Epoch 2/5
938/938 [==============================] - 18s 19ms/step - loss: 0.0587 - accuracy: 0.9816
Epoch 3/5
938/938 [==============================] - 18s 19ms/step - loss: 0.0392 - accuracy: 0.9877
Epoch 4/5
938/938 [==============================] - 18s 19ms/step - loss: 0.0309 - accuracy: 0.9908
Epoch 5/5
938/938 [==============================] - 18s 19ms/step - loss: 0.0244 - accuracy: 0.9927


In [73]:
# Get accuracy from max and avg models from their respective history data
max_accuracy = max_history.history['accuracy']
avg_accuracy = avg_history.history['accuracy']
# Define range of epochs for use in 
epochs = range(1, num_epochs + 1)
# Plot max and average accuracy against number of epochs using matplotlib
plt.plot(epochs, max_accuracy, label='Max')
plt.plot(epochs, avg_accuracy, label='Avg')
# Add title, labels, and legend
plt.title('Accuracy for Max and Average Pooling')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

NameError: name &#39;plt&#39; is not defined

In [ ]:
# The plot above shows that the maxpooling model achieves a higher overall accuracy and has a higher increase at each epoch number
# This is understandable. In book: TODO

In [ ]:
# PROBLEM 3
# Define results dict for storing accuracy values
results_max = {}
# Define all filer sizes for use in loop
filter_sizes = [ 
    ('3x3',(3,3)),
    ('4x4',(4,4)),
    ('5x5',(5,5)),
    ('6x6',(6,6))
]
# Increase number of epochs to make plot slightly more clear
epochs = range(1, num_epochs + 1)
# Loop through all filtersizes and train a model with maxpooling layers
for filter_name, filter_size in filter_sizes:
    model = models.Sequential([
        Conv2D( 32, filter_size, activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    # Compile and fit model according to assignment instructions, retaining history for later recall
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_images, train_labels, epochs=num_epochs, batch_size=64)
    # Get accuracy from history and add to results dict
    accuracy = history.history['accuracy']
    results_max[filter_name] = accuracy

In [ ]:
# Clear previous plot
plt.clf()
# Increase size of plot for clarity
plt.rcParams["figure.figsize"] = (10,10)
# Each key-value pair in the results dict is the name of the kernel size and a list of accuracy values at each epoch, respectively.
for key, val in results_max.items():
    # Plot accuracy against number of epochs
    plt.plot(epochs, val, label=key)
# Add title, labels, and legend
plt.title('Accuracy for Kernel Sizes')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# This plot shows that the model with the 3x3 kernel trains a bit slower, but ultimately has the highest accuracy
# The model with the 5x5 kernel learns the fastest but is the second worst.
# The model with the 4x4 kernel appears to be converging with the 3x3 at epoch 7, which is not shown here, as I'm keeping the number of epochs consistent for use later. But I did this calculation previously.

In [ ]:
# PROBLEM 4
# Define results dict for storing accuracy values
results_padding = {}
# Define model with zero padding layer before each conv2d layer
padding_model = models.Sequential([
    ZeroPadding2D(padding=(3,3)),
    Conv2D( 32, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    ZeroPadding2D(padding=(3,3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    ZeroPadding2D(padding=(3,3)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
# Compile and fit model according to assignment instructions, retaining history for later recall
padding_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
padding_model_history = padding_model.fit(train_images, train_labels, epochs=num_epochs, batch_size=64)
# Get accuracy and add to results dict
accuracy = padding_model_history.history['accuracy']
results_padding['padding'] = accuracy

In [ ]:
# Define model without padding layers added
no_padding_model = models.Sequential([
    Conv2D( 32, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
# Compile and fit model according to assignment instructions, retaining history for later recall
no_padding_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
no_padding_model_history = no_padding_model.fit(train_images, train_labels, epochs=num_epochs, batch_size=64)
# Get accuracy and add to results dict
accuracy = no_padding_model_history.history['accuracy']
results_padding['no_padding'] = accuracy

In [ ]:
# Add accuracy of max model in problem 2
results_padding['original'] = max_accuracy

In [ ]:
# Clear plot
plt.clf()
epochs = range(1, num_epochs + 1)
print(epochs)
# Increase size of plot for clarity
plt.rcParams["figure.figsize"] = (10,10)
# Loop through padding results dict
# Each item is the key (name) and value (list of accuracy values)
for key, val in results_padding.items():
    # Plot accuracy against number of epochs
    plt.plot(epochs, val, label=key)
# Add title, labels, and legend
plt.title('Accuracy for Padding Models')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# This plot shows that the model with padding achieves higher accuracy overall and with less training. The second best is the original and not padding is the worst in the end, though that one trains a little faster.

In [ ]:
# PROBLEM 5
# Define url for cat picture
url = 'https://cdn.cnn.com/cnnnext/dam/assets/200713171600-02-silk-road-cats-exlarge-169.jpg'
# Get image at url using request module and open using PIL
img = Image.open(request.urlopen(url))
# Define size, kernel, scale, and offset of image
size = (3,3)
kernel = (-1, -1, -1, -1, 8, -1, -1, -1, -1)
scale = 1
offset = 0
# Define laplace filter, using PIL ImageFilter
laplace_filter = ImageFilter.Kernel( size=size, kernel=kernel, scale=scale, offset=offset)
# Make img with edges detected using laplace filter
img_edge = img.filter(laplace_filter)

In [ ]:
# Clear plot
plt.clf()
# Use subplots to get plots side by side
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5))
# Show first image
ax1.imshow(img)
# Remove ticks and set title
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title('Original')

# Show second image
ax2.imshow(img_edge)
# Remove ticks and set title
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_title('After Processing with Edge Detection Filter')
fig.tight_layout()